In [1]:
### testing agents

In [1]:
from mb_rag.agents.run_agent import agent_runner_functions

/home/winnow/.local/lib/python3.10/site-packages/langchain/chains/api/base.py:56: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import TextRequestsWrapper
/home/winnow/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

In [3]:
agent_runner_functions?

Object `agent_runner_functions` not found.


In [4]:
my_agent = agent_runner_functions()

NameError: name 'agent_runner_functions' is not defined

In [8]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool
from typing import Type
import os
import tavily
class SimpleSearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class SimpleSearchTool(BaseTool):
    name = "simple_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SimpleSearchInput

    def _run(
        self,
        query: str,
    ) -> str:
        """Use the tool."""
        from tavily import TavilyClient

        api_key = os.getenv("TAVILY_API_KEY")
        client = TavilyClient(api_key=api_key)
        results = client.search(query=query)
        return f"Search results for: {query}\n\n\n{results}\n"


In [9]:
tools = [SimpleSearchTool()]

In [10]:
my_agent.tools(tools)

<bound method agent_runner_functions.tools of <mb_rag.agents.run_agent.agent_runner_functions object at 0x75696bb7ddd0>>

In [26]:
agent_c = my_agent.create_agent(tools=tools)

In [28]:
agent_exec =my_agent.agent_executor(agent_c,tools=tools)

In [29]:
agent_exec.invoke({"input":"what is the weather in london?"})



> Entering new AgentExecutor chain...

Invoking: `simple_search` with `{'query': 'current weather in London'}`


Search results for: current weather in London


{'query': 'current weather in London', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in London', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.52, 'lon': -0.11, 'tz_id': 'Europe/London', 'localtime_epoch': 1725337114, 'localtime': '2024-09-03 05:18'}, 'current': {'last_updated_epoch': 1725336900, 'last_updated': '2024-09-03 05:15', 'temp_c': 17.1, 'temp_f': 62.8, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 200, 'wind_dir': 'SSW', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 100, 'f

{'input': 'what is the weather in london?',
 'output': 'The current weather in London is as follows:\n\n- **Temperature**: 17.1°C (62.8°F)\n- **Condition**: Overcast\n- **Wind**: 3.8 mph (6.1 kph) from the SSW\n- **Humidity**: 94%\n- **Visibility**: 10 km (6 miles)\n- **Pressure**: 1011.0 mb\n- **UV Index**: 1 (Low)\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).'}

In [31]:
res =my_agent.agent_invoke(agent_exec,"what is the weather in london?")



> Entering new AgentExecutor chain...

Invoking: `simple_search` with `{'query': 'current weather in London'}`


Search results for: current weather in London


{'query': 'current weather in London', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in London', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.52, 'lon': -0.11, 'tz_id': 'Europe/London', 'localtime_epoch': 1725337114, 'localtime': '2024-09-03 05:18'}, 'current': {'last_updated_epoch': 1725336900, 'last_updated': '2024-09-03 05:15', 'temp_c': 17.1, 'temp_f': 62.8, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 200, 'wind_dir': 'SSW', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 100, 'f

In [34]:
res

{'input': 'what is the weather in london?',
 'output': 'The current weather in London is as follows:\n\n- **Temperature**: 17.1°C (62.8°F)\n- **Condition**: Overcast\n- **Wind**: 3.8 mph (6.1 kph) from the SSW\n- **Humidity**: 94%\n- **Visibility**: 10 km (6 miles)\n- **Pressure**: 1011.0 mb\n- **UV Index**: 1\n\nIt is currently nighttime in London.'}

In [33]:
res["output"]

'The current weather in London is as follows:\n\n- **Temperature**: 17.1°C (62.8°F)\n- **Condition**: Overcast\n- **Wind**: 3.8 mph (6.1 kph) from the SSW\n- **Humidity**: 94%\n- **Visibility**: 10 km (6 miles)\n- **Pressure**: 1011.0 mb\n- **UV Index**: 1\n\nIt is currently nighttime in London.'